# Import Libraries

In [1]:
!pip install evalml

     |████████████████████████████████| 6.3 MB 5.1 MB/s 
     |████████████████████████████████| 142 kB 58.7 MB/s 
     |████████████████████████████████| 327 kB 54.6 MB/s 
     |████████████████████████████████| 101 kB 9.4 MB/s 
     |████████████████████████████████| 80 kB 8.6 MB/s 
     |████████████████████████████████| 18.0 MB 25 kB/s 
     |████████████████████████████████| 11.3 MB 58.4 MB/s 
     |████████████████████████████████| 3.4 MB 45.6 MB/s 
     |████████████████████████████████| 22.3 MB 30 kB/s 
     |████████████████████████████████| 2.0 MB 51.6 MB/s 
     |████████████████████████████████| 356 kB 62.1 MB/s 
     |████████████████████████████████| 23.9 MB 14 kB/s 
     |████████████████████████████████| 296 kB 60.7 MB/s 
     |████████████████████████████████| 5.9 MB 59.0 MB/s 
     |████████████████████████████████| 28.5 MB 50 kB/s 
     |████████████████████████████████| 9.8 MB 47.1 MB/s 
     |████████████████████████████████| 1.6 MB 69.3 MB/s 
     |███████████████

In [1]:
import pandas as pd
import numpy as np
import evalml
from evalml.objectives import get_core_objectives
from evalml.problem_types import ProblemTypes
from evalml.data_checks import DefaultDataChecks
from evalml.automl import AutoMLSearch
from sklearn.utils import shuffle
import pickle
import warnings
warnings.filterwarnings('ignore')

# Read Datasets

In [2]:
df = pd.read_csv('final.csv')
df.head()

,Family_History,Patient_History,Patient_Sex,Patient_Age,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,Melanoma
0,1,1,1,1.374412,0.000000,0.000000,0.000000,0.0,0.895281,0.170424,0.334496,0.535144,0.000000,0.838951,1.313835,0.0,0.799786,0.848957,0.766729,0.072602,0.000000,0.744514,0.271474,0.427653,0.0,0.225046,0.898221,0.105062,0.486301,0.332734,0.989699,0.000000,1.105003,0.000000,0.000000,0.243165,0
1,1,0,1,0.785378,0.428445,0.808372,0.410599,0.0,0.000000,0.782431,0.000000,0.000000,0.201100,0.000000,1.692971,0.0,0.000000,0.085503,1.626967,0.000000,0.524208,1.524392,0.000000,0.000000,0.0,0.000000,0.895318,0.389831,0.000000,1.209965,1.254844,0.467543,0.012432,0.611834,0.765858,0.598891,1
2,1,0,1,0.785378,0.671592,1.086340,0.713988,0.0,0.000000,0.988801,0.000000,0.000000,0.371980,0.000000,1.817110,0.0,0.000000,0.000000,1.996230,0.000000,0.578080,1.840052,0.000000,0.000000,0.0,0.000000,1.517236,0.362477,0.000000,1.636171,1.432802,0.800722,0.000000,1.061281,0.950372,0.683324,1
3,1,1,0,0.883551,0.615322,0.877099,0.626662,0.0,0.000000,1.120432,0.000000,0.000000,0.544277,0.000000,1.498737,0.0,0.000000,0.000000,1.821136,0.000000,0.370582,1.988797,0.000000,0.000000,0.0,0.000000,1.588848,0.510777,0.000000,1.399091,1.189754,0.802155,0.000000,1.460947,1.134553,0.553451,1
4,1,1,1,0.981723,0.212209,0.627659,0.242865,0.0,0.000000,0.614922,0.000000,0.014604,0.175750,0.000000,1.565679,0.0,0.000000,0.316299,1.365290,0.000000,0.292966,1.309489,0.000000,0.001812,0.0,0.000000,0.751302,0.281071,0.000000,0.987256,1.237673,0.329191,0.143877,0.644764,0.543582,0.544338,1


In [3]:
for i in range(100):
  df = shuffle(df)

df=df.reset_index(drop=True)

In [4]:
# Dataframe to DataTable conversion to treat columns with the same physical data type differently
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
X_train,_, y_train,_ = evalml.preprocessing.split_data(X, y, 
                                                      problem_type='binary',
                                                      test_size=1/df.shape[0]*100)

# Training

In [5]:
for objective in get_core_objectives(ProblemTypes.BINARY):
    print(objective.name)

MCC Binary
Log Loss Binary
Gini
AUC
Precision
F1
Balanced Accuracy Binary
Accuracy Binary


In [6]:
data_checks = DefaultDataChecks("binary", "Balanced Accuracy Binary")
data_checks.validate(X_train, y_train)

{'actions': [{'code': 'DROP_COL', 'metadata': {'column': 'Family_History'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'Patient_History'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'Patient_Sex'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'feature_3'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'feature_4'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'feature_6'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'feature_7'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'feature_9'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'feature_11'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'feature_12'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'feature_15'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'feature_18'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'feature_19'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'feature_20'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'feature_21'}},
  {'code': 'DROP_COL', 'metadata': {'column': 'feature

In [7]:
automl = AutoMLSearch(X_train = X_train, 
                      y_train = y_train, 
                      problem_type = 'binary',
                      objective = "Balanced Accuracy Binary",
                      additional_objectives=['Log Loss Binary','F1','AUC'],
                      optimize_thresholds = True,
                      ensembling = True,
                      max_batches = 10,
                      verbose=True
                 )
automl.search()

Generating pipelines to search over...
8 pipelines ready for search.
Ensembling will run every 9 batches.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Balanced Accuracy Binary. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 10 batches for a total of 51 pipelines. 
Allowed model families: decision_tree, lightgbm, linear_model, extra_trees, random_forest, xgboost, catboost



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean Balanced Accuracy Binary: 0.500

*****************************
* Evaluating Batch Number 1 *
*****************************

Elastic Net Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Balanced Accuracy Binary: 0.707
Decision Tree Classifier w/ Imputer:
	Starting cross validation
	Finished cross validation - mean Balanced Accuracy Binary: 0.613
Random Forest Classifier w/ Imputer:
	Starting cross validation
	Finished cross validation - mean Balanced Accuracy Binary: 0.700
LightGBM Classifier w/ Imputer:
	Starting cross validation
	Finished cross validation - mean Balanced Accuracy Binary: 0.659
Logistic Regression Classifier w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Balanced Accuracy Binary: 0.714
XGBoost C

In [8]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])


**********************************
* CatBoost Classifier w/ Imputer *
**********************************

Problem Type: binary
Model Family: CatBoost

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. CatBoost Classifier
	 * n_estimators : 10
	 * eta : 0.03
	 * max_depth : 6
	 * bootstrap_type : None
	 * silent : True
	 * allow_writing_files : False
	 * n_jobs : -1

Training
Training for binary problems.
Objective to optimize binary classification pipeline thresholds for: <evalml.objectives.standard_metrics.BalancedAccuracyBinary object at 0x7f16ce390950>
Total training time (including CV): 2.1 seconds

Cross Validation
----------------
             Balanced Accuracy Binary  Log Loss Binary    F1   AUC # Training # Validation
0                               0.712            0.568 0.509 0.744        771          386
1                               0.694         

In [9]:
automl.best_pipeline.graph_feature_importance()